In [1]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm


In [17]:
### Function to Commodities Data
# Define the list of commodity tickers
#tickers = ['GC=F',  # Gold futures
#           'SI=F',  # Silver futures
#           'CL=F',  # Crude oil futures
#           'HG=F',  # Copper futures
#           'NG=F']  # Natural gas futures
tickers = ['GC=F']

def import_commod_data(tickers, start_date):
    data = pd.DataFrame()
    if len(tickers) == 1:
        data[tickers[0]] = yf.download(tickers[0], start_date)['Adj Close']
    else:
        for t in tickers:
            data[t] = yf.download(t, start_date)['Adj Close']
    return data

start_date = '2022-01-01'
commod_data = import_commod_data(tickers, start_date)
commod_data = commod_data.reset_index() # Turn Multi-Index into Column in Pandas 
commod_data.tail()

[*********************100%%**********************]  1 of 1 completed


,Date,GC=F
585,2024-05-01,2299.899902
586,2024-05-02,2299.199951
587,2024-05-03,2299.000000
588,2024-05-06,2321.600098
589,2024-05-07,2321.699951
